# Housing Regression Maching Learning Project

### Imports (Data and Packages)

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from data_preprocessing_final import cleaning, dummify_func
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
%matplotlib inline 

In [28]:
housing = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)
housing = housing[housing['SaleCondition'] == 'Normal']
real_estate = pd.read_csv('./data/Ames_Real_Estate_data.csv' , index_col = 0)

C:\Users\jackc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
housing = cleaning(housing)
input, target = dummify_func(housing)
target = np.log(target)

In [5]:
input.shape

(2413, 324)

### EDA

In [6]:
## See correlations related to sale price and to eyeball which features might matter the most 
corrs = housing.corr()
highly_correlated = corrs[np.abs(corrs['SalePrice']) >= .6]
medium_correlated = corrs[np.abs(corrs['SalePrice']).between(.4,.6)] 
low_correlated = corrs[(np.abs(corrs['SalePrice']) < .4)] 

## Print out column headers that are going to have high and medium correlations to sales price 
print('Highly correlated variables are: ')
print(highly_correlated['SalePrice'])
print('-'*150)
print('Medium correlated variables are: ')
print(medium_correlated['SalePrice'])
print('-'*150)
print('Low correlated variables are: ')
print(low_correlated['SalePrice'])

Highly correlated variables are: 
GrLivArea      0.736040
SalePrice      1.000000
OverallQual    0.787266
TotalBsmtSF    0.642555
1stFlrSF       0.634755
GarageCars     0.632217
GarageArea     0.621855
Name: SalePrice, dtype: float64
------------------------------------------------------------------------------------------------------------------------------------------------------
Medium correlated variables are: 
YearBuilt       0.533163
YearRemodAdd    0.501543
MasVnrArea      0.486528
BsmtFinSF1      0.454164
FullBath        0.548532
TotRmsAbvGrd    0.496571
Fireplaces      0.493448
GarageYrBlt     0.492190
Name: SalePrice, dtype: float64
------------------------------------------------------------------------------------------------------------------------------------------------------
Low correlated variables are: 
PID             -0.207391
MSSubClass      -0.075707
LotFrontage      0.344709
LotArea          0.275087
OverallCond     -0.091298
BsmtFinSF2       0.038297
BsmtUnfSF  

### Start dummifying variables and look at multicollinearity

In [7]:
## Identify Categorical Variables and create a list of them (With PID included so we can use as key later on)
category_list = list(housing.select_dtypes('object').columns)
category_list.append('PID')

## Use the list above to create a dataframe with just the categorical variables and PID
category_variables = housing[housing.columns.intersection(category_list)]

## Dummify the categorical variables
category = pd.get_dummies(data = category_variables, prefix_sep = '_' , drop_first=True)

## Remove the original categorical values from the dataframe and merge the dummified and original values into a new dataframe called final
housing.drop(columns = housing.select_dtypes('object').columns, axis =1, inplace=True)
final = pd.merge(housing, category, on = 'PID')


## Begin work on Lasso Regression

In [8]:
## Working on Lasso Regression
from sklearn.linear_model import Lasso
lasso = Lasso()

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(input,target,test_size=0.2)

In [54]:
## Find the right Alpha 

train_R2 = []
test_R2  = []
alphaRange = np.linspace(0,.1,100)
for alpha in alphaRange:
    lasso.set_params(alpha=alpha, normalize=False, max_iter=2000)  
    lasso.fit(X_train, Y_train)
    train_R2.append(lasso.score(X_train, Y_train))
    test_R2.append(lasso.score(X_test, Y_test))

<ipython-input-54-77d44aed3a85>:8: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(X_train, Y_train)
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.70879063714203, tolerance: 0.026515730385325835
  model = cd_fast.enet_coordinate_descent(


In [14]:
lasso_pd = pd.DataFrame(lassoCoef)

In [65]:
lasso_pd.reset_index(inplace=True)
lasso_pd.columns = ['Feature', 'Lasso_coef']

ValueError: Length mismatch: Expected axis has 3 elements, new values have 2 elements

In [64]:
# lasso_pd.sort_values(by = ['Lasso_coef', 'Feature'], ascending=False)

In [17]:
print('Training = ', lasso.score(X_train, Y_train))
print('Testing = ', lasso.score(X_test, Y_test))

Training =  0.9443360314734589
Testing =  0.9399558276091852


In [63]:
features_removed = lasso_pd[lasso_pd['Lasso_coef'] == 0]
features_removed.shape

(168, 2)

In [62]:
features_removed.shape[0]

168

In [25]:
## Uncomment to see which features were removed using Lasso
pd.set_option('display.max_rows', None)
features_removed.sort_values(by = "Feature")

,Feature,Lasso_coef
6,2ndFlrSF,0.0
17,3SsnPorch,0.0
28,Alley_Pave,0.0
84,BldgType_2fmCon,-0.0
156,BsmtCond_Fa,-0.0
159,BsmtCond_Po,-0.0
168,BsmtFinType1_No_Basement,-0.0
173,BsmtFinType2_LwQ,0.0
174,BsmtFinType2_No_Basement,-0.0
176,BsmtFinType2_Unf,-0.0


## Trying Grid Search

In [58]:
alphaRange = np.linspace(0,0.002,5)


params = {'alpha': alphaRange, 'max_iter' : [10000]}

In [59]:
gs = GridSearchCV(lasso, params, scoring = 'r2')
gs.fit(X_train, Y_train)

C:\Users\jackc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.191179837690762, tolerance: 0.020980515008368002
  model = cd_fast.enet_coordinate_descent(
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimat

GridSearchCV(estimator=Lasso(alpha=0.1, max_iter=2000),
             param_grid={'alpha': array([0.    , 0.0005, 0.001 , 0.0015, 0.002 ]),
                         'max_iter': [10000]},
             scoring='r2')

In [49]:
gs.best_params_

{'alpha': 0.0005, 'max_iter': 10000}

In [60]:
gs.score(X_test, Y_test)

0.9146953059733863

In [61]:
gs.score(X_train, Y_train)

0.938078243561895